In [1]:
import pandas as pd
import numpy as np

%matplotlib inline

In [2]:
# PROBLEM STATEMENT 1.1
df = pd.read_csv('Small_Cap/PVR.csv')

In [3]:
df.head()

,Symbol,Series,Date,Prev Close,Open Price,High Price,Low Price,Last Price,Close Price,Average Price,Total Traded Quantity,Turnover,No. of Trades,Deliverable Qty,% Dly Qt to Traded Qty
0,PVR,EQ,15-May-2017,1526.20,1532.35,1538.15,1515.45,1520.00,1524.95,1526.51,103693,1.582883e+08,7359,54518,52.58
1,PVR,EQ,16-May-2017,1524.95,1526.70,1545.00,1526.65,1533.25,1539.00,1538.04,86547,1.331129e+08,6332,39020,45.09
2,PVR,EQ,17-May-2017,1539.00,1539.70,1552.85,1526.05,1527.00,1530.30,1538.01,73902,1.136622e+08,4665,21303,28.83
3,PVR,EQ,18-May-2017,1530.30,1525.00,1539.35,1502.20,1514.90,1515.05,1519.68,82125,1.248035e+08,2903,38796,47.24
4,PVR,EQ,19-May-2017,1515.05,1515.00,1534.50,1502.20,1517.65,1513.55,1513.25,114890,1.738567e+08,9495,54764,47.67


In [4]:
# PROBLEM STATEMENT 1.2
# Calculate the max., min. and the mean price of the stock in the last 90 trading days
df_last_90_days = df.tail(90)
last_90_close = df_last_90_days['Close Price']

min_90 = last_90_close.min()
max_90 = last_90_close.max()
mean_90 = last_90_close.mean()

print('min : ' + str(min_90))
print('max : ' + str(max_90))
print('mean : ' + str(mean_90))

min : 1443.4
max : 1808.1
mean : 1615.5588888888888


In [5]:
# PROBLEM STATEMENT 1.3
# Change the dtype of the 'Date' column to datetime

import datetime as dt

df['Date'] = df['Date'].apply(lambda x : dt.datetime.strptime(x, '%d-%b-%Y'))

# Subtracting min value of 'Date' from max value
print(df['Date'].max() - df['Date'].min())

728 days 00:00:00


In [6]:
# PROBLEM STATEMENT 1.4
# Calculate monthwise VWAP of the stock.

df['Month'] = df['Date'].apply(lambda x : dt.datetime.strftime(x, '%B'))
df['Year'] = df['Date'].apply(lambda x : dt.datetime.strftime(x, '%Y'))

# To calculate the VWAP we will also add a 'PV' columns 
# which gives us the "price * volume" for that day
df['PV'] = df['Close Price'] * df['Total Traded Quantity']

# Calculate VWAP
df_group = df.groupby(['Year', 'Month']).sum()

VWAP = df_group['PV'] / df_group['Total Traded Quantity'] 
print(VWAP)

Year  Month    
2017  August       1332.542259
      December     1370.179452
      July         1382.736564
      June         1480.541916
      May          1483.152422
      November     1355.812855
      October      1372.506699
      September    1276.299095
2018  April        1332.177906
      August       1263.282944
      December     1540.634877
      February     1367.896232
      January      1463.548441
      July         1178.151493
      June         1380.027513
      March        1266.689726
      May          1397.760378
      November     1422.715683
      October      1276.412981
      September    1339.018074
2019  April        1704.549861
      February     1510.185194
      January      1601.108687
      March        1617.733294
      May          1771.578334
dtype: float64


In [7]:
# PROBLEM STATEMENT 1.5
# (a) Write a function to calculate average stock price over last N days
# (b) Write a funciton to calculate profit/loss percentage over last N days

# Note : The values will be calculated assuming the current date to be the last date in the dataset (i.e. 13-May-2019)

# Function (a)
def avg_over_days(N):
    delta = dt.timedelta(days = N)
    df_filtered = df[df['Date'] >= df.loc[df.shape[0]-1]['Date'] - delta]['Close Price']
    return df_filtered.mean()

# Function (b)
def profit_loss_perc_over_days(N):
    delta = dt.timedelta(days = N)
    df_filtered = df_filtered = df[df['Date'] >= df.iloc[-1]['Date'] - delta]['Close Price']
    
    # using formula : 100 * (final_price - initial_price) / initial_price
    return 100 * (df_filtered.iloc[-1] - df_filtered.iloc[0]) / df_filtered.iloc[0]


print('Avg. price over 7 days : {0}, Profit % : {1}'.format(avg_over_days(7), profit_loss_perc_over_days(7)))
print('Avg. price over 2 weeks : {0}, Profit % : {1}'.format(avg_over_days(14), profit_loss_perc_over_days(14)))
print('Avg. price over 1 month : {0}, Profit % : {1}'.format(avg_over_days(30), profit_loss_perc_over_days(30)))
print('Avg. price over 3 months : {0}, Profit % : {1}'.format(avg_over_days(90), profit_loss_perc_over_days(90)))
print('Avg. price over 6 months : {0}, Profit % : {1}'.format(avg_over_days(180), profit_loss_perc_over_days(180)))
print('Avg. price over 1 year : {0}, Profit % : {1}'.format(avg_over_days(365), profit_loss_perc_over_days(365)))

Avg. price over 7 days : 1754.7916666666667, Profit % : -4.3444487599178805
Avg. price over 2 weeks : 1767.9222222222222, Profit % : -2.568594761083955
Avg. price over 1 month : 1738.0823529411766, Profit % : 1.4445850127982556
Avg. price over 3 months : 1626.237288135593, Profit % : 18.49611657158567
Avg. price over 6 months : 1584.2714876033056, Profit % : 21.43410579699936
Avg. price over 1 year : 1445.5670731707316, Profit % : 21.940868581128875


In [8]:
# PROBLEM STATEMENT 1.6
# Add column 'Day_perc_change'

df['Day_Perc_Change'] = df['Close Price'].pct_change() * 100

# set the first value to 0 because it is NaN. It is NaN because 2 values are required to
# calculate the percentage but only 1 value is available for the first row
df.at[0,'Day_Perc_Change'] = 0

In [9]:
# PROBLEM STATEMENT 1.7
# Add a 'Trend' column to the dataset

def get_trend(change_perc):
    if change_perc < 0.5 and change_perc >= -0.5:
        return 'slight or no change'
    elif change_perc >= 0.5 and change_perc < 1:
        return 'slight positive'
    elif change_perc < -0.5 and change_perc >= -1:
        return 'slight negative'
    elif change_perc >= 1 and change_perc < 3:
        return 'positive'
    elif change_perc < -1 and change_perc >= -3:
        return 'negative'
    elif change_perc >= 3 and change_perc < 7:
        return 'among top gainers'
    elif change_perc < 3 and change_perc >= -7:
        return 'among top losers'
    elif change_perc >= 7:
        return 'bull run'
    elif change_perc < -7:
        return 'bear drop'
    
df['Trend'] = df['Day_Perc_Change'].apply(lambda x : get_trend(x))

In [10]:
# PROBLEM STATEMENT 1.8
# Find mean and median of 'Total Traded Quantity' for each category of 'Trend'
print('MEAN : ')
print(df.groupby('Trend').mean()['Total Traded Quantity'])

print('\n\n MEDIAN :')
print(df.groupby('Trend').median()['Total Traded Quantity'])

MEAN : 
Trend
among top gainers      6.426534e+05
among top losers       4.835644e+05
bear drop              2.042056e+06
bull run               6.050980e+05
negative               3.159358e+05
positive               3.835129e+05
slight negative        2.353158e+05
slight or no change    2.494382e+05
slight positive        2.616963e+05
Name: Total Traded Quantity, dtype: float64


 MEDIAN :
Trend
among top gainers       472254.0
among top losers        285156.0
bear drop              2042055.5
bull run                605098.0
negative                267615.0
positive                301429.0
slight negative         180868.5
slight or no change     203996.0
slight positive         222040.5
Name: Total Traded Quantity, dtype: float64


In [11]:
# PROBLEM STATEMENT 1.9
# Save the results

df.to_csv('week2.csv', index=False)